### MNIST Classification using 2 layer CNN

In [1]:
#loading modules 
import numpy as np
import tensorflow as tf


/home/armughan/anaconda2/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/armughan/anaconda2/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#hyper parameteres
#following architecture INPUT -> [[CONV -> RELU]*M -> POOL?]*N -> [FC -> RELU]*K -> FC
params={}
params['M']=1
params['N']=2
params['K']=1
params['use_bn']=True
params['conv_layer_1_1']={'kernel':5,"stride":1,'padding':'same','filters':32}
params['pool_layer_1']={'pool_size':2,'strides':2}
params['conv_layer_1_2']={'kernel':5,"stride":1,'padding':'same','filters':64}
params['pool_layer_2']={'pool_size':2,'strides':2}
params['dense_layer_1']={'units':1024}
params['dropout_rate']=0.4
params['learning_rate']=0.001
params['num_classes']=10


In [6]:
def complex_cnn(features,labels,mode,params):

    
    M=params.get('M',1)
    N=params.get('N',1)
    K=params.get('K',1)
    use_bn=params.get('use_bn',False)
    lr=params.get('learning_rate',0.001)
    num_classes=params['num_classes']
    dropout_rate=params.get('dropout_rate',0)
    
    inputs=tf.reshape(features['x'],[-1,28,28,1])
#     model={} #contains tensorflow variables
    
    spatial_dims=np.asarray([28,28])
    filters=1
    
    for i in range(N):
        
        for j in range(M):
            conv_params=params.get('conv_layer_'+str(j+1)+'_'+str(i+1),{})
            
            filters=conv_params.get('filters',1)
            kernel_size=conv_params.get('kernel',5)
            strides=conv_params.get('stride',1)
            padding=conv_params.get('padding','same')
            name='conv_layer_'+str(j+1)+'_'+str(i+1)
            
            if(padding=='valid'):
                spatial_dims=((spatial_dims-np.asarray(kernel_size))//np.asarray(strides))+1
                
            with tf.name_scope('conv_layer_'+str(j+1)+'_'+str(i+1)):
                inputs=tf.layers.conv2d(
                    inputs=inputs,
                    filters=filters,
                    kernel_size=kernel_size,
                    strides=strides,
                    padding=padding,
                    name=name,
                    activation=tf.nn.relu)

                w = tf.get_variable(name+"/kernel")
                tf.summary.histogram(' weights',w)
                tf.summary.histogram(' activations',inputs)
            
            
#             inputs=model['conv_layer_'+str(j+1)+'_'+str(i+1)]
        
        
        pool_params=params.get('pool_layer_'+str(i+1),{})  
        pool_size=pool_params.get('pool_size',[2,2])
        strides= pool_params.get('strides',2)
        
        with tf.name_scope('pool_layer_'+str(i)):
            inputs=tf.layers.max_pooling2d(
                inputs=inputs,
                pool_size=pool_size,
                strides=strides)
#         inputs=model['pool_layer_'+str(i+1)]
        
        spatial_dims=((spatial_dims-np.asarray(pool_size))//np.asarray(strides))+1
            
            
    flat_inputs=tf.reshape(inputs, [-1,np.prod(spatial_dims)*filters])
#     flat_inputs=tf.reshape(inputs, [-1,7*7*64])
    
    for i in range(K):
        dense_params=params.get('dense_layer_'+str(i+1),{})
        with tf.name_scope('dense_layer_'+str(i)):
            flat_inputs=tf.layers.dense(
                inputs=flat_inputs,
                units=dense_params.get('units',1024), 
                activation=tf.nn.relu)
        with tf.name_scope('dense_dropout_layer_'+str(i)):
            flat_inputs=tf.layers.dropout(inputs=flat_inputs,
                                        rate=dropout_rate, 
                                      training=mode == tf.estimator.ModeKeys.TRAIN)
        
#         inputs=model['dense_layer_'+str(i+1)]
        # Logits Layer
    with tf.name_scope('logits_layer'):
        logits = tf.layers.dense(inputs=flat_inputs, units=num_classes)
        
    
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    with tf.name_scope('total'):
        loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
        tf.summary.scalar('loss',loss)
    

    merged=tf.summary.merge_all()
    train_wr
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
        
        
        

In [4]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [5]:
#Create the Estimator
mnist_classifier = tf.estimator.Estimator(model_fn=complex_cnn, model_dir="/tmp/mnist_convnet_model",params=params)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f23c5da0b38>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=10,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    hooks=[logging_hook])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-3456
INFO:tensorflow:Saving checkpoints for 3457 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.00055055 0.00029034 0.5011116  0.00135732 0.10172775 0.00040841
  0.33611262 0.000921   0.02950942 0.02801087]
 [0.00193561 0.853315   0.0105707  0.01770282 0.00771018 0.05061927
  0.01443013 0.01286417 0.02836337 0.0024887 ]
 [0.00000829 0.00000733 0.00000392 0.00137642 0.00000675 0.00003782
  0.00000007 0.9976526  0.00015877 0.00074807]
 [0.00806591 0.00065868 0.00011104 0.00465647 0.13446379 0.05982642
  0.00852394 0.603286   0.121011   0.05939664]
 [0.00026877 0.927749   0.00627798 0.01570551 0.00409488 0.00412374
  0.00327841 0.00816581 0.02182171 0.00851431]
 [0.00000664 0.00000385 0.00001205 0.00022203 0.9864205  0.00009607
  0.00004903 0.00389262 0.00044918 0.00884817]
 [0.00883438 0.00193678 0.00239525 0.14095709 0.02696421 0.313820

INFO:tensorflow:loss = 0.4051455, step = 3457
INFO:tensorflow:probabilities = [[0.84989285 0.00000241 0.00013844 0.00182609 0.00000285 0.14522137
  0.00071158 0.0001551  0.00158377 0.00046556]
 [0.00197213 0.00000954 0.0000021  0.00390099 0.00094013 0.01388841
  0.00000522 0.7374439  0.00117354 0.24066402]
 [0.00066223 0.00089091 0.00284292 0.01429748 0.7807287  0.00242112
  0.00388406 0.04680689 0.01369069 0.13377498]
 [0.72709507 0.00059727 0.01193102 0.03064883 0.00021268 0.04789847
  0.0368433  0.00180156 0.13745554 0.00551622]
 [0.02454129 0.00001547 0.00119116 0.00146097 0.05996218 0.7914919
  0.00197638 0.00931554 0.05490614 0.05513895]
 [0.01455938 0.00111364 0.02911445 0.03943444 0.00436678 0.16102955
  0.04573397 0.00010636 0.70156085 0.00298059]
 [0.00126669 0.00153277 0.5408423  0.01800834 0.00042815 0.00865027
  0.00209979 0.31953752 0.10543035 0.0022039 ]
 [0.00012994 0.00299821 0.3267182  0.18525718 0.00056899 0.01760359
  0.000463   0.26823366 0.16323666 0.03479054]
 [0

INFO:tensorflow:global_step/sec: 2.62816
INFO:tensorflow:probabilities = [[0.00000096 0.00000016 0.00181609 0.00015247 0.8940757  0.00030973
  0.00116866 0.00157629 0.00292643 0.09797345]
 [0.00002661 0.00001127 0.9969785  0.00037581 0.00001191 0.00004523
  0.00189398 0.00000262 0.00065114 0.00000283]
 [0.00001234 0.00000153 0.00011574 0.00011004 0.9488323  0.00012486
  0.00021382 0.00273359 0.00067858 0.04717724]
 [0.9996438  0.         0.0000023  0.00000053 0.         0.00033694
  0.00000406 0.0000002  0.00001205 0.00000011]
 [0.00000038 0.00102466 0.15880936 0.83279705 0.00001106 0.00092985
  0.00008067 0.00039092 0.00566722 0.00028877]
 [0.00000226 0.0076989  0.98668206 0.0006502  0.00001176 0.00006782
  0.00050935 0.00000094 0.0043669  0.00000987]
 [0.00014435 0.00678243 0.00291039 0.03912083 0.0144735  0.03212976
  0.00100842 0.02200632 0.86590123 0.0155227 ]
 [0.00192468 0.00009233 0.00772361 0.9325576  0.00001679 0.01228425
  0.00001066 0.0029617  0.03780245 0.00462594]
 [0.010

INFO:tensorflow:loss = 0.5435524, step = 3557 (38.059 sec)
INFO:tensorflow:probabilities = [[0.01104667 0.00451983 0.00120425 0.02290171 0.00338613 0.7868457
  0.00035264 0.11002286 0.03718791 0.02253232]
 [0.00019103 0.91329217 0.01172856 0.00406524 0.00059543 0.00558944
  0.01900864 0.00107459 0.04434751 0.00010741]
 [0.00032134 0.00072858 0.02897142 0.0008766  0.21188462 0.00049676
  0.7225449  0.00920806 0.00863595 0.01633162]
 [0.00123674 0.00662702 0.03340748 0.03081866 0.11080959 0.01518648
  0.00527178 0.02505116 0.5659598  0.2056313 ]
 [0.00000939 0.00018015 0.00000329 0.00010588 0.0000213  0.00120969
  0.00000046 0.9790228  0.00880729 0.01063975]
 [0.07312603 0.00003965 0.00084473 0.00000809 0.00088589 0.86992747
  0.00041913 0.01375826 0.03213133 0.00885939]
 [0.00229781 0.00001856 0.00117422 0.00017099 0.03080894 0.0061374
  0.00252145 0.03166578 0.00211121 0.9230937 ]
 [0.00088486 0.00255481 0.00133132 0.00417202 0.00474848 0.00435128
  0.00012173 0.91919595 0.02897107 0.0

INFO:tensorflow:global_step/sec: 2.61997
INFO:tensorflow:probabilities = [[0.00033599 0.00000014 0.00003304 0.00054861 0.01329992 0.0001997
  0.0000139  0.3079053  0.00745221 0.67021114]
 [0.9944232  0.         0.00000954 0.00006368 0.00000047 0.00525089
  0.000241   0.00000041 0.00000943 0.00000146]
 [0.00273354 0.00032625 0.03265079 0.03143389 0.01588998 0.01279584
  0.7718899  0.00063764 0.12958087 0.0020614 ]
 [0.05560011 0.00000018 0.00000022 0.00040056 0.00024045 0.02520127
  0.00002304 0.8566287  0.00063402 0.06127137]
 [0.01237645 0.00002525 0.00269938 0.22279446 0.00001787 0.6994438
  0.00203237 0.00002501 0.06038989 0.0001955 ]
 [0.0000095  0.00000192 0.00000208 0.00004672 0.00000606 0.00005025
  0.00000019 0.9959162  0.00008348 0.00388358]
 [0.00104539 0.9292166  0.02197187 0.00570748 0.00189372 0.00278067
  0.00989037 0.00537899 0.01795185 0.00416308]
 [0.00040898 0.00000747 0.00013102 0.97814834 0.0000061  0.00674561
  0.0000014  0.00371149 0.0099506  0.00088906]
 [0.00097

INFO:tensorflow:loss = 0.38196182, step = 3657 (38.170 sec)
INFO:tensorflow:probabilities = [[0.00013548 0.0000186  0.00158533 0.99217814 0.00000013 0.00033104
  0.00000246 0.00540196 0.00030693 0.00003997]
 [0.0000129  0.99467397 0.00037796 0.00142629 0.00012252 0.00005641
  0.00041489 0.00125108 0.00096397 0.00069986]
 [0.00044648 0.9654214  0.00329083 0.00184693 0.00023877 0.00137057
  0.01633962 0.00071897 0.00940603 0.0009205 ]
 [0.00035395 0.00000001 0.00000004 0.00035573 0.00306655 0.00107637
  0.00000076 0.9474619  0.00040298 0.04728172]
 [0.00007112 0.00000022 0.0000006  0.00006083 0.00020657 0.9991974
  0.00003438 0.00001079 0.00041382 0.00000431]
 [0.000094   0.00000137 0.0000001  0.00009387 0.00043717 0.00218549
  0.00000074 0.88982177 0.00026666 0.10709889]
 [0.00095029 0.00120244 0.96074075 0.00994334 0.00001975 0.00358202
  0.01047786 0.00006434 0.01298839 0.00003083]
 [0.0002457  0.98182    0.00108419 0.00145086 0.0000821  0.00017048
  0.0004318  0.00041085 0.01377188 0

INFO:tensorflow:global_step/sec: 2.74342
INFO:tensorflow:probabilities = [[0.00738864 0.00948307 0.00404215 0.08812581 0.0026318  0.6062793
  0.00151262 0.08490302 0.13330217 0.06233136]
 [0.00381831 0.00007802 0.03358671 0.00388448 0.16139151 0.17315108
  0.08011985 0.0010678  0.12588775 0.4170145 ]
 [0.00262612 0.09769027 0.04770674 0.00818364 0.00106798 0.00366319
  0.00386409 0.00313136 0.82318133 0.00888531]
 [0.00010647 0.00000016 0.00000147 0.00005265 0.96575767 0.00479933
  0.00099302 0.00119724 0.01497632 0.01211567]
 [0.00015042 0.00000446 0.00000351 0.00022168 0.06930352 0.0012921
  0.00016046 0.00335145 0.00322844 0.92228395]
 [0.99957377 0.         0.00004927 0.00000084 0.         0.00032483
  0.00004921 0.00000003 0.00000206 0.00000001]
 [0.00000197 0.0000001  0.00000044 0.00004008 0.00000218 0.00000265
  0.00000004 0.9988231  0.00004463 0.00108473]
 [0.00012077 0.00031403 0.00601678 0.00020403 0.00131469 0.00102213
  0.98718077 0.00002209 0.00365372 0.00015096]
 [0.00000

INFO:tensorflow:loss = 0.32652298, step = 3757 (36.460 sec)
INFO:tensorflow:probabilities = [[0.0006019  0.00484441 0.01981142 0.000228   0.01627163 0.00052097
  0.9514121  0.0006341  0.00134716 0.00432832]
 [0.0029417  0.00044668 0.00184864 0.00429473 0.26069468 0.00800517
  0.00219575 0.05951731 0.0133145  0.64674085]
 [0.0515966  0.00001096 0.00009206 0.00123272 0.00159984 0.03288741
  0.00019299 0.65084195 0.00906541 0.25248012]
 [0.00101901 0.00042642 0.8691591  0.00076063 0.05174764 0.00041373
  0.07030396 0.00007196 0.00419675 0.00190094]
 [0.00315465 0.00078577 0.05410168 0.00085374 0.83613676 0.00113646
  0.03904647 0.00760381 0.0040125  0.05316817]
 [0.00001094 0.00002719 0.0000668  0.9916875  0.00000206 0.00471034
  0.0000012  0.00022403 0.00195475 0.00131534]
 [0.99078923 0.         0.00000945 0.00014124 0.         0.00891483
  0.00000091 0.0000861  0.00004983 0.00000844]
 [0.0012733  0.8695235  0.04664615 0.00457698 0.00294536 0.00565706
  0.0156978  0.00332536 0.04506442 

KeyboardInterrupt: 

In [ ]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)